# sort sections
- task 4: sections of original data are shuffled
- task 5: Perform experiment after sorting sections

## read section order

In [ ]:
def read_section_order(path):
    section_order = {}
    path_section_o = path
    file = open(path_section_o, "r")
    lines = file.readlines()
    
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        if line=="":
            continue
        doctype = line.split("\t")[0]
        section = line.split("\t")[1]
        is_asmt = line.split("\t")[2]
        
        section_order[doctype+"\t"+section] = l
    return section_order

section_order = read_section_order("./data/02_type_to_file_links/sections_visits_ordered_2011to2020.txt")
print("section_order: ", section_order)

## read samples

In [ ]:
import glob, os

def sort_samples(datapath = "./data/05_sampled/train", mode = "train"):
    files = glob.glob(datapath+"/*.txt")
    files.sort()

    for f in range(len(files)):
        file = open(files[f], "r")
        lines = file.readlines()
        file.close()

        filename = files[f].split("/")[-1]

        pt_id = files[f].split("/")[-1]
    #     print("pt_id: ", pt_id)

        dict_lines = {}

        for l in range(len(lines)):
            line = lines[l].strip("\n")
    #         print("line: ", line)

            docid = line.split("\t")[0]
            rowid = line.split("\t")[1]
            doctype = line.split("\t")[2]
            section = line.split("\t")[3]
            label = line.split("\t")[4]
            content = line.split("\t")[5]

            section_id = section_order[doctype+"\t"+section]

            zeros_s = "".join(['0']*(8-len(str(section_id))))
            zeros_r = "".join(['0']*(8-len(str(rowid))))
            key = str(docid)+"\t"+str(zeros_s)+str(section_id)+"\t"+zeros_r+str(rowid)

            # values
            # doctype \t section \t label \t content
            values = doctype+"\t"+section +"\t"+ label+"\t"+content

            dict_lines[key] = values
        
        # sort sections
        dict_lines_sort = dict(sorted(dict_lines.items()))

        outdata = []
        sent_id = 0
        docid_last = -1
        for key in dict_lines_sort:
            docid = key.split("\t")[0]
            section_id = key.split("\t")[1]
            rowid = key.split("\t")[2]
            
            if docid!=docid_last:
                sent_id = 0
            
            values = dict_lines_sort[key].split("\t")
            doctype = values[0]
            section = values[1]
            label   = values[2]
            content = values[3]
            
            outdata.append(docid+"\t"+str(sent_id)+"\t"+doctype+"\t"+section+"\t"+label+"\t"+content)
            
            sent_id = sent_id + 1
            docid_last = docid
            

        outdir = "./data/05_sampled_sorted/"+str(mode)
        if not os.path.exists(outdir):
            os.makedirs(outdir)

        file = open(outdir+"/"+str(filename), "w")
        file.write("\n".join(outdata))
        file.close()
        
    
    print("Done")


sort_samples(datapath = "./data/05_sampled/train", mode = "train")
sort_samples(datapath = "./data/05_sampled/test", mode = "test")

